# Provisioning AIO Cluster with K3D

This guide walks you through the steps to provision an Azure IoT Operations (AIO) cluster using the provided Bicep template. The Bicep template is based on the example available in an Azure IoT Operations repository.

## Azure Resource Dependencies

1. Azure Key Vault
1. Azure Arc
1. Service Principal

## Prerequisites

1. A running GitHub Codespace of this repository.
1. Dev container setup with k3d cluster. Upon creation of the Codespace, a k3d cluster "devcluster" is already running and ready to use.
1. **Azure Subscription**: `az login` has been executed and default subscription has been set. Ensure you have access to an Azure subscription where you can deploy resources.

If required, reset your environment using the [00-clean-up.sh](./00-clean-up.sh) script, which will delete the k3d cluster and then recreate the cluster with the right image and settings. 

In [27]:
./00-clean-up.sh

Azure resource group 'rg-emm03084' is being deleted
K3D registry deleted
INFO[0000] Deleting cluster 'devcluster'                
INFO[0007] Deleting cluster network 'k3d-devcluster'    
INFO[0007] Deleting 1 attached volumes...               
INFO[0007] Removing cluster details from default kubeconfig... 
INFO[0007] Removing standalone kubeconfig file (if there is one)... 
INFO[0007] Successfully deleted cluster devcluster!     
INFO[0000] Creating node 'k3d-devregistry.localhost'    
INFO[0000] Successfully created registry 'k3d-devregistry.localhost' 
INFO[0000] Starting Node 'k3d-devregistry.localhost'    
INFO[0000] Successfully created registry 'k3d-devregistry.localhost' 
# You can now use the registry like this (example):
# 1. create a new cluster that uses this registry
k3d cluster create --registry-use k3d-devregistry.localhost:5500

# 2. tag an existing local image to be pushed to the registry
docker tag nginx:latest k3d-devregistry.localhost:5500/mynginx:v0.1

# 3. push tha

## Steps

### 1. Connect to Azure Arc
Set the params as env variables. Ensure that for the parameter `LOCATION` you use one of the [supported regions](https://learn.microsoft.com/en-us/azure/iot-operations/get-started/quickstart-deploy?tabs=linux#connect-a-kubernetes-cluster-to-azure-arc).


In [30]:
export RESOURCE_GROUP="rg-aio2"
export CLUSTER_NAME="mycluster"
export LOCATION="westus2"

Run the following script to connect the k3d Kubernetes cluster to Azure Arc.

In [31]:
./01-arc-connect.sh

ARC_CUSTOMLOCATION_OID is not set. Setting it to the value of LOCATION westus2.


{
  "id": "/subscriptions/20d5770a-8bb2-402f-9ade-11d91cb478a2/resourceGroups/rg-aio2",
  "location": "westus2",
  "managedBy": null,
  "name": "rg-aio2",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null,
  "type": "Microsoft.Resources/resourceGroups"
}
  "id": "/subscriptions/20d5770a-8bb2-402f-9ade-11d91cb478a2/resourceGroups/rg-aio2",
  "location": "westus2",
  "managedBy": null,
  "name": "rg-aio2",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null,
  "type": "Microsoft.Resources/resourceGroups"
}
NAME                      STATUS   ROLES                  AGE     VERSION
k3d-devcluster-server-0   Ready    control-plane,master   2m50s   v1.25.3+k3s1
This operation might take a while...

The required pre-checks for onboarding have succeeded.
Azure resource provisioning has begun.
Azure resource provisioning has finished.
Starting to install Azure arc agents on the Kubernetes cluster.
{
  "agentPublicKeyCertificate": "MIICCgKCAgEAyVKn2

### 3. Create service principal
Run the script to create a service principal with an app name of your choice.

In [33]:
./02-create-sp.sh aio2

Creating Service Principal ...


Save the app ID and client secret of the created service principal and set as env variables. These will be in `~/.azure/servicePrincipal.json`. Set the object ID as well.

In [34]:
export AKV_SP_CLIENT_ID=$(cat ~/.azure/servicePrincipal.json | jq -r '.clientId')
export AKV_SP_CLIENT_SECRET=$(cat ~/.azure/servicePrincipal.json | jq -r '.clientSecret')
export AKV_SP_OBJECT_ID=$(az ad sp show --id $AKV_SP_CLIENT_ID --query id -o tsv)

### 4. Deploy the Bicep Template

Run the following script to provision the Azure Key Vault and deploy AIO with MQ. 

The script automatically sets the Azure Key Vault name based on the Resource Group name you set to `$RESOURCE_GROUP` variable and a random number to prevent name conflicts.

> **Note**: The deployment will take a few minutes to complete. To see the output of the `03-aio-deploy-core.sh` script, look in the `output.log` file created in the same directory.

In [35]:
nohup ./03-aio-deploy-core.sh > output.log 2>&1 & tail -f output.log | tail -n 1

[1] 247579


In [ ]:
./04-aio-deploy-bicep.sh && ./05-aio-deploy-aks.sh

### 5. Verify Deployment

Once the deployment is complete, you can verify the following k8s extensions in your cluster are in the Azure Portal with the status `Succeeded`:

- azure-iot-operations
- mq

### 6. Setup the observability stack (optional)

The observability stack is a set of Azure resources including Azure Monitor, Grafana, and Prometheus connections to the cluster to gather cluster insights, traces and metrics. Run the following script from the provisioning folder:

In [9]:
./06-observability.sh

== Enabling observability ==
Command group 'config' is experimental and under development. Reference and support levels: https://aka.ms/CLI_refstatus
Creating an Azure Monitor Logs Analytics Workspace ...
{
  "accountId": "87504d18-991b-40b9-8dee-62b6d6b32072",
  "defaultIngestionSettings": {
    "dataCollectionEndpointResourceId": "/subscriptions/20d5770a-8bb2-402f-9ade-11d91cb478a2/resourceGroups/MA_monitor-mycluster_westus2_managed_9/providers/Microsoft.Insights/dataCollectionEndpoints/monitor-mycluster",
    "dataCollectionRuleResourceId": "/subscriptions/20d5770a-8bb2-402f-9ade-11d91cb478a2/resourceGroups/MA_monitor-mycluster_westus2_managed_9/providers/Microsoft.Insights/dataCollectionRules/monitor-mycluster"
  },
  "etag": "\"7301664e-0000-0800-0000-65eb98f90000\"",
  "id": "/subscriptions/20d5770a-8bb2-402f-9ade-11d91cb478a2/resourcegroups/rg-emm03082/providers/microsoft.monitor/accounts/monitor-mycluster",
  "location": "westus2",
  "metrics": {
    "internalId": "mac_87504d18

For more information about using the observability stack, go to [OBSERVABILITY.md](../../docs/OBSERVABILITY.md).

### 7. Testing MQ Broker

1. Open the MQTT UI in a terminal:

   ```bash
   mqttui
   ```

1. Open a new Bash terminal and publish a message.

In [10]:
mqttui publish "in-cluster-topic" 'hello from within the pod'

1. Verify you have received the published message in the first terminal you opened the MQTT UI. For that, use the MQTT UI navigation instructions to navigate to the `in-cluster-topic`.

    ![in-cluster-topic in MQTTUI](../../docs/assets/mqttui-in-cluster-topic.png)

### 8. OPCUA Messages Simulator

You can provision the simulator into the system with:

In [ ]:
./08-simulator.sh

This will create a pod (data-simulator) that will constantly run and send messages every 5 seconds with the two payloads of Good and MachineStatus.

You can open MQTTUI to visualize all the messages flowing in this order:

1. opcua-simulator
1. opcua
1. input
1. input/valid or input/invalid
1. current-shift-total
1. metrics/aio/machine-status and metrics/aio/total-count

![MQTTUI All Topics](../../docs/assets/mqttui-alltopics.png)


## Next Steps

Deploy the data processing pipelines in [infra/deployment](../deployment/DEPLOYMENT.ipynb)